#### Note: Celery Crumble was listed as H/NH ? Giger bread man listed as M/F?

#### Note: for now we exclued 'Ten in the Bed and Other Counting Rhymes' (but if includig, there is a mismatch in secondary characters).

#### Question: how do we want to handle groups containing multiple named characters? e.g. see The Three Little Pigs

In [118]:
from openai import OpenAI

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [119]:
# TO_EXCLUDE = ['Mini Beasties', 'Ten in the Bed and Other Counting Rhymes', 'The Rescue Party', 'One Snowy Night', 'The Night Before Christmas', 'One Starry Night'] # Books that currently has issues in the excel file
TO_EXCLUDE = ['Ten in the Bed and Other Counting Rhymes']

In [120]:
labels = pd.read_excel('./Book-List-Final-NONA.xlsx', sheet_name='Sheet1')
# labels = pd.read_excel('/home/rustybilges/Downloads/Book-List-Final-NONA.xlsx', sheet_name='Sheet1')
labels = labels.rename(columns={'Author ': 'Author'})
for exclude in TO_EXCLUDE:
    labels = labels[labels.Title != exclude]

In [121]:
labels

,Number,Title,Author,Author Gender,Cost,Source,Starting Page,Ending Page,Direct Address,Protagonist Gender,Protagonist Human/ Non-human,Protagonist Name,Secondary Character Gender,SC Human/Non Human,SC Names,Notes,Year of First Publication
0,1.0,Seasons,John Burningham,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This book costs 40/50 pounds to buy and is not in libraries,NaN
1,2.0,Owl Babies,Martin Waddell,M,2.75,ebay,6.0,29.0,N,F/M/M,NH,Sarah/Percy/Bill,F,NH,Owl Mother,NaN,1992.0
2,3.0,Where's My Teddy,Jez Alborough,M,NaN,NaN,6.0,29.0,N,M,H,Eddy,M/NGS/NGS,NH/NH/NH,Freddy; teddy/giant teddy;giant teddy bear/gigantic bear;the bear,NaN,1992.0
3,4.0,We're Going on a Bear Hunt,Michael Rosen and Helen Oxenbury,M/F,2.79,ebay,5.0,37.0,N,NGS,H,we,NGS,NH,A Bear,NaN,1989.0
4,5.0,Gruff the Grump,Steve Smallman and Cee Biscoe,M/F,3.05,ebay,9.0,30.0,N,M,NH,Mr Bear; Gruff the Grump,F,NH,rabbit,"Pretty gendered - protagonist is male, big, scowly, grizzly etc... rabbit is female, he rescues her she pleads and cries and gives him presents. Now I type all that it's pretty awful but I actually quite liked it on first read... :/",2009.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,NaN,NaN,Average cost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
protagonists = {
    'book': [],
    'name': [],
    'gender': [],
    'human': [],
    'alias_list': [],
    'alias_count': []
}

In [123]:
pd.set_option('display.max_colwidth', None)
labels[labels.Title == 'The Three Little Pigs'][['Protagonist Name', 'Protagonist Gender']]

,Protagonist Name,Protagonist Gender
200,mother pig/first little pig;pigs/second little pig;pigs/third little pig;pigs/man carrying a large bundle of straw/man carrying a bundle of sticks/man pulling a cart laden with heavy bricks,F/M/M/M/M/M/M


In [124]:
for ri, row in labels.iterrows():

    if not pd.isna(row['Protagonist Name']):
        print(row.Title)
        
        protagonist_name_list = row['Protagonist Name'].split('/')
        protagonist_gender_list = row['Protagonist Gender'].split('/')
        
        # Human flag not recorded for first person protagonist:
        if pd.isna(row['Protagonist Human/ Non-human']):
            protagonist_human_list = ['H' for i in protagonist_name_list]
        elif row.Title == 'The Monstrous Tale of Celery Crumble':
            protagonist_human_list = ['H']
        else:
            protagonist_human_list = row['Protagonist Human/ Non-human'].split('/')
        
        if row.Title == 'The Gingerbread Man':
            protagonist_gender_list = ['M']
        # The gender and human flags are only entered once if they are the same for all characters.
        # We handle this here:
        if len(protagonist_name_list) != len(protagonist_gender_list):
            assert len(protagonist_gender_list) == 1
            protagonist_gender_list = [protagonist_gender_list[0] for i in protagonist_name_list]
            
        
        if len(protagonist_name_list) != len(protagonist_human_list):
            assert len(protagonist_human_list) == 1
            protagonist_human_list = [protagonist_human_list[0] for i in protagonist_name_list]

        for ni, name in enumerate(protagonist_name_list):
            protagonists['book'].append(row.Title)
            protagonists['gender'].append(protagonist_gender_list[ni])
            protagonists['human'].append(protagonist_human_list[ni])
            
            if len(name.split(';')) > 1:
                # map aliases
                print(f"Mapping aliases for: [{name}]")
                aliases = name.split(';')
                protagonists['name'].append(aliases[0])
                protagonists['alias_count'].append(len(aliases) - 1)
                protagonists['alias_list'].append(aliases[1:])
            else:
                protagonists['name'].append(name)
                protagonists['alias_count'].append(0)
                protagonists['alias_list'].append([])

Owl Babies
Where's My Teddy
We're Going on a Bear Hunt
Gruff the Grump
Mapping aliases for: [Mr Bear; Gruff the Grump]
The Owl's Lesson
The Fox's Hiccups
The Hedgehog's Balloon
The Rescue Party
The Cross Rabbit
One Snowy Night
After the Storm
Can't You Sleep Little Bear
Father Christmas Needs A Wee
The Polar Express
Mapping aliases for: [I;We]
How The Grinch Stole Christmas
Santa is Coming to Devon
The First Christmas
A Letter to Santa
The Night Before Christmas
Jesus' Christmas Party
One Starry Night
The Christmas Story
Mapping aliases for: [her baby;little dove]
Elmer and Grandpa Eldo
Elmer and the Stranger
Elmer and Wilbur
Elmer in the Snow
Elmer and the Lost Teddy
Elmer and the Wind
Elmer on Stilts
Elmer and Rose
Cinder the Bubble-Blowing Dragon
All For One
Where's My Cuddle
Sir Charlie Stinky Socks and the Really Big Adventure
Who Will Sing My Puff-a-bye
Harry and the Dinosaurs say Raahh!
Harry and the Dinosaurs at the Museum
Harry and the Bucketful of Dinosaurs
Romp in the Swamp


In [125]:
# protagonists = pd.DataFrame(protagonists)
protagonists

{'book': ['Owl Babies',
  'Owl Babies',
  'Owl Babies',
  "Where's My Teddy",
  "We're Going on a Bear Hunt",
  'Gruff the Grump',
  "The Owl's Lesson",
  "The Fox's Hiccups",
  "The Hedgehog's Balloon",
  'The Rescue Party',
  'The Cross Rabbit',
  'One Snowy Night',
  'After the Storm',
  "Can't You Sleep Little Bear",
  "Can't You Sleep Little Bear",
  'Father Christmas Needs A Wee',
  'The Polar Express',
  'How The Grinch Stole Christmas',
  'Santa is Coming to Devon',
  'The First Christmas',
  'A Letter to Santa',
  'The Night Before Christmas',
  "Jesus' Christmas Party",
  'One Starry Night',
  'The Christmas Story',
  'The Christmas Story',
  'Elmer and Grandpa Eldo',
  'Elmer and the Stranger',
  'Elmer and Wilbur',
  'Elmer in the Snow',
  'Elmer and the Lost Teddy',
  'Elmer and the Wind',
  'Elmer on Stilts',
  'Elmer and Rose',
  'Cinder the Bubble-Blowing Dragon',
  'All For One',
  "Where's My Cuddle",
  'Sir Charlie Stinky Socks and the Really Big Adventure',
  'Who W

#### Now we do the same to extract secondary characters:

In [126]:
secondary_characters = {
    'book': [],
    'name': [],
    'gender': [],
    'human': [],
    'alias_list': [],
    'alias_count': []
}

In [127]:
for ri, row in labels.iterrows():

    if not pd.isna(row['SC Names']):
        print(row.Title)
        
        protagonist_name_list = row['SC Names'].split('/')
        protagonist_gender_list = row['Secondary Character Gender'].split('/')
        
        # Human flag not recorded for first person protagonist:
        if pd.isna(row['SC Human/Non Human']):
            protagonist_human_list = ['H' for i in protagonist_name_list]
#         elif row.Title == 'The Monstrous Tale of Celery Crumble':
#             protagonist_human_list = ['H']
        else:
            protagonist_human_list = row['SC Human/Non Human'].split('/')
        
        print(protagonist_name_list)
        print(protagonist_gender_list)
        print(protagonist_human_list)
        print(len(protagonist_name_list))
        print(len(protagonist_gender_list))
        print(len(protagonist_human_list))
#         if row.Title == 'The Gingerbread Man':
#             protagonist_gender_list = ['M']
        # The gender and human flags are only entered once if they are the same for all characters.
        # We handle this here:
        if len(protagonist_name_list) != len(protagonist_gender_list):
            assert len(protagonist_gender_list) == 1
            protagonist_gender_list = [protagonist_gender_list[0] for i in protagonist_name_list]
            
        
        if len(protagonist_name_list) != len(protagonist_human_list):
            assert len(protagonist_human_list) == 1
            protagonist_human_list = [protagonist_human_list[0] for i in protagonist_name_list]

        for ni, name in enumerate(protagonist_name_list):
            secondary_characters['book'].append(row.Title)
            secondary_characters['gender'].append(protagonist_gender_list[ni])
            secondary_characters['human'].append(protagonist_human_list[ni])
            
            if len(name.split(';')) > 1:
                # map aliases
                print(f"Mapping aliases for: [{name}]")
                aliases = name.split(';')
                secondary_characters['name'].append(aliases[0])
                secondary_characters['alias_count'].append(len(aliases) - 1)
                secondary_characters['alias_list'].append(aliases[1:])
            else:
                secondary_characters['name'].append(name)
                secondary_characters['alias_count'].append(0)
                secondary_characters['alias_list'].append([])

Owl Babies
['Owl Mother']
['F']
['NH']
1
1
1
Where's My Teddy
['Freddy; teddy', 'giant teddy;giant teddy bear', 'gigantic bear;the bear']
['M', 'NGS', 'NGS']
['NH', 'NH', 'NH']
3
3
3
Mapping aliases for: [Freddy; teddy]
Mapping aliases for: [giant teddy;giant teddy bear]
Mapping aliases for: [gigantic bear;the bear]
We're Going on a Bear Hunt
['A Bear']
['NGS']
['NH']
1
1
1
Gruff the Grump
['rabbit']
['F']
['NH']
1
1
1
The Owl's Lesson
['Squirrel', 'mole', 'little bird', 'owl']
['M', 'NGS', 'M', 'F']
['NH']
4
4
1
The Fox's Hiccups
['fox; Mr ghost']
['M ']
['NH']
1
1
1
Mapping aliases for: [fox; Mr ghost]
The Hedgehog's Balloon
['hedgehog', 'mice;mouse']
['M', 'NGS']
['NH']
2
2
1
Mapping aliases for: [mice;mouse]
The Rescue Party
['two brothers;his brother;two rabbits', 'little sister;smallest rabbit;little rabbit', 'fox', 'three young rabbits;the rabbits;another rabbit;his sibblings', 'the squirrels', 'the rescuers']
['M', 'F', 'NGS', 'NGS', 'NGS', 'NGS']
['NH']
6
6
1
Mapping aliases f

In [128]:
# secondary_characters = pd.DataFrame(secondary_characters)

In [129]:
secondary_characters

{'book': ['Owl Babies',
  "Where's My Teddy",
  "Where's My Teddy",
  "Where's My Teddy",
  "We're Going on a Bear Hunt",
  'Gruff the Grump',
  "The Owl's Lesson",
  "The Owl's Lesson",
  "The Owl's Lesson",
  "The Owl's Lesson",
  "The Fox's Hiccups",
  "The Hedgehog's Balloon",
  "The Hedgehog's Balloon",
  'The Rescue Party',
  'The Rescue Party',
  'The Rescue Party',
  'The Rescue Party',
  'The Rescue Party',
  'The Rescue Party',
  'The Cross Rabbit',
  'The Cross Rabbit',
  'One Snowy Night',
  'One Snowy Night',
  'One Snowy Night',
  'One Snowy Night',
  'One Snowy Night',
  'One Snowy Night',
  'One Snowy Night',
  'One Snowy Night',
  'One Snowy Night',
  'After the Storm',
  'After the Storm',
  'After the Storm',
  'After the Storm',
  'After the Storm',
  'After the Storm',
  'Father Christmas Needs A Wee',
  'Father Christmas Needs A Wee',
  'Father Christmas Needs A Wee',
  'The Polar Express',
  'The Polar Express',
  'The Polar Express',
  'The Polar Express',
  'Th

In [130]:
all_characters = {
    key: protagonists[key] + secondary_characters[key]
    for key in protagonists.keys()
}

In [131]:
all_characters = pd.DataFrame(all_characters)
all_characters['name'] = all_characters['name'].str.strip()

In [135]:
import sqlite3

In [136]:
conn = sqlite3.connect('character_database.db')

In [137]:
all_characters[['book', 'name', 'gender', 'human', 'alias_count']].to_sql('characters', conn, if_exists='replace')

In [138]:
temp = pd.read_sql('select * from characters', conn, index_col='index')

In [139]:
with_aliases = all_characters[all_characters.alias_count > 0]

In [140]:
alias_table = {
    'alias': [],
    'character': [],
    'character_id': [],
    'book': []
}

In [141]:
for ri, row, in with_aliases.iterrows():
    for alias in row.alias_list:
        alias_table['alias'].append(alias)
        alias_table['book'].append(row.book)
        alias_table['character'].append(row['name'])
        alias_table['character_id'].append(ri)

# Manually adding The Troll which was missed during coding
alias_table['alias'].append('The Troll')
alias_table['book'].append('The Troll')
alias_table['character'].append('Troll')
alias_table['character_id'].append(50)

In [142]:
alias_table = pd.DataFrame(alias_table)

In [143]:
alias_table.to_sql('aliases', conn, if_exists='replace')

In [144]:
alias_table = pd.read_sql('select * from aliases', conn, index_col='index')

In [145]:
alias_table

,alias,character,character_id,book
index,,,,
0,Gruff the Grump,Mr Bear,5,Gruff the Grump
1,We,I,16,The Polar Express
2,little dove,her baby,24,The Christmas Story
3,Hany,Harry,43,Harry and the Dinosaurs Go Wild
4,McTat,Tabby McTat,51,Tabby McTat
...,...,...,...,...
197,everyone,other children,1429,Eleanor Won't Share
198,mother,mother duck,1481,The Ugly Duckling
199,the swans,white birds,1483,The Ugly Duckling


In [146]:
len(all_characters.book.unique())

193

In [147]:
len(all_characters.name.unique())

905